# Лабораторна робота №2: Аналіз VHI
Мета: завантаження даних з NOAA, аналіз вегетаційного індексу (VHI) по областях України.

In [ ]:
import urllib.request
import pandas as pd
import re
from datetime import datetime
from io import StringIO
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
region_names = {
    1: "Вінницька", 2: "Волинська", 3: "Дніпропетровська", 4: "Донецька",
    5: "Житомирська", 6: "Закарпатська", 7: "Запорізька", 8: "Івано-Франківська",
    9: "Київська", 10: "Кіровоградська", 11: "Луганська", 12: "Львівська",
    13: "Миколаївська", 14: "Одеська", 15: "Полтавська", 16: "Рівненська",
    17: "Сумська", 18: "Тернопільська", 19: "Харківська", 20: "Херсонська",
    21: "Хмельницька", 22: "Черкаська", 23: "Чернівецька", 24: "Чернігівська",
    25: "м.Київ"
}

def clean_noaa_file(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        content = f.read()

    match = re.search(r'<tt><pre>(.*?)</pre>', content, re.DOTALL)
    if not match:
        return pd.DataFrame()

    table_text = match.group(1).strip()
    lines = [line.rstrip(', ').strip() for line in table_text.splitlines() if line.strip()]
    data_str = '\n'.join(lines)

    df = pd.read_csv(
        StringIO(data_str),
        sep=r',\s*',
        engine='python',
        names=['year', 'week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI']
    )

    df = df[df['year'].astype(str).str.match(r'^\d{4}$', na=False)]
    df['year'] = df['year'].astype(int)
    df['week'] = df['week'].astype(int)
    df['VHI'] = pd.to_numeric(df['VHI'], errors='coerce')
    df = df[df['VHI'].notna() & (df['VHI'] != -1)]
    return df